In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.5
pandas 1.0.4
sklearn 0.23.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [2]:
!ls generate_csv

test_00.csv  test_08.csv   train_06.csv  train_14.csv  valid_02.csv
test_01.csv  test_09.csv   train_07.csv  train_15.csv  valid_03.csv
test_02.csv  train_00.csv  train_08.csv  train_16.csv  valid_04.csv
test_03.csv  train_01.csv  train_09.csv  train_17.csv  valid_05.csv
test_04.csv  train_02.csv  train_10.csv  train_18.csv  valid_06.csv
test_05.csv  train_03.csv  train_11.csv  train_19.csv  valid_07.csv
test_06.csv  train_04.csv  train_12.csv  valid_00.csv  valid_08.csv
test_07.csv  train_05.csv  train_13.csv  valid_01.csv  valid_09.csv


In [3]:
source_dir = "./generate_csv/"

#通过判断开头去添加文件
def get_filenames_by_prefix(source_dir, prefix_name):
    all_files = os.listdir(source_dir)
    results = []
    for filename in all_files:
        if filename.startswith(prefix_name):
            results.append(os.path.join(source_dir, filename))
    return results

train_filenames = get_filenames_by_prefix(source_dir, "train")
valid_filenames = get_filenames_by_prefix(source_dir, "valid")
test_filenames = get_filenames_by_prefix(source_dir, "test")

import pprint
pprint.pprint(train_filenames)
pprint.pprint(valid_filenames)
pprint.pprint(test_filenames)


['./generate_csv/train_08.csv',
 './generate_csv/train_11.csv',
 './generate_csv/train_18.csv',
 './generate_csv/train_15.csv',
 './generate_csv/train_17.csv',
 './generate_csv/train_00.csv',
 './generate_csv/train_01.csv',
 './generate_csv/train_19.csv',
 './generate_csv/train_14.csv',
 './generate_csv/train_02.csv',
 './generate_csv/train_16.csv',
 './generate_csv/train_09.csv',
 './generate_csv/train_03.csv',
 './generate_csv/train_12.csv',
 './generate_csv/train_10.csv',
 './generate_csv/train_13.csv',
 './generate_csv/train_05.csv',
 './generate_csv/train_07.csv',
 './generate_csv/train_04.csv',
 './generate_csv/train_06.csv']
['./generate_csv/valid_01.csv',
 './generate_csv/valid_05.csv',
 './generate_csv/valid_02.csv',
 './generate_csv/valid_04.csv',
 './generate_csv/valid_08.csv',
 './generate_csv/valid_07.csv',
 './generate_csv/valid_06.csv',
 './generate_csv/valid_00.csv',
 './generate_csv/valid_09.csv',
 './generate_csv/valid_03.csv']
['./generate_csv/test_00.csv',
 './gener

In [4]:
#下面的接口都是之前用过的
def parse_csv_line(line, n_fields = 9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x, y

def csv_reader_dataset(filenames, n_readers=5,
                       batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_csv_line,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32
train_set = csv_reader_dataset(train_filenames,
                               batch_size = batch_size)
valid_set = csv_reader_dataset(valid_filenames,
                               batch_size = batch_size)
test_set = csv_reader_dataset(test_filenames,
                              batch_size = batch_size)


In [5]:
!ls

chapter_4.tar.gz	   tf02_data_generate_csv.ipynb
generate_csv		   tf03-tfrecord_basic_api.ipynb
temp.csv		   tf04_data_generate_tfrecord.ipynb
tf01-data_basic_api.ipynb  tfrecord_basic


In [6]:
#把基础的如何序列化的步骤搞到一个函数
def serialize_example(x, y):
    """Converts x, y to tf.train.Example and serialize"""
    input_feautres = tf.train.FloatList(value = x)
    label = tf.train.FloatList(value = y)
    features = tf.train.Features(
        feature = {
            "input_features": tf.train.Feature(
                float_list = input_feautres),
            "label": tf.train.Feature(float_list = label)
        }
    )
    #把features变为example
    example = tf.train.Example(features = features)
    return example.SerializeToString()  #把example序列化
#n_shards是存为多少个文件，steps_per_shard和 steps_per_epoch类似
def csv_dataset_to_tfrecords(base_filename, dataset,
                             n_shards, steps_per_shard,
                             compression_type = None):
    #压缩文件类型
    options = tf.io.TFRecordOptions(
        compression_type = compression_type)
    all_filenames = []
    
    for shard_id in range(n_shards):
        filename_fullpath = '{}_{:05d}-of-{:05d}'.format(
            base_filename, shard_id, n_shards)
        #打开文件
        with tf.io.TFRecordWriter(filename_fullpath, options) as writer:
            #取出数据,为什么skip，上一个文件写了前500行，下一个文件存后面的数据
            for x_batch, y_batch in dataset.skip(shard_id * steps_per_shard).take(steps_per_shard):
                for x_example, y_example in zip(x_batch, y_batch):
                    writer.write(
                        serialize_example(x_example, y_example))
        all_filenames.append(filename_fullpath)
    #返回所有tfrecord文件名
    return all_filenames

In [11]:
!rm -rf generate_tfrecords

In [8]:
for i in train_set.take(1):
    print(i)  

(<tf.Tensor: shape=(32, 8), dtype=float32, numpy=
array([[ 4.2408210e-01,  9.1296333e-01, -4.4374820e-02, -1.5297213e-01,
        -2.4727628e-01, -1.0539167e-01,  8.6126745e-01, -1.3357790e+00],
       [ 4.0127665e-01, -9.2934215e-01, -5.3330503e-02, -1.8659453e-01,
         6.5456617e-01,  2.6434466e-02,  9.3125278e-01, -1.4406418e+00],
       [-1.1199750e+00, -1.3298433e+00,  1.4190045e-01,  4.6581370e-01,
        -1.0301778e-01, -1.0744184e-01, -7.9505241e-01,  1.5304717e+00],
       [ 4.9710345e-02, -8.4924191e-01, -6.2146995e-02,  1.7878747e-01,
        -8.0253541e-01,  5.0660671e-04,  6.4664572e-01, -1.1060793e+00],
       [ 6.3034356e-01,  1.8741661e+00, -6.7132145e-02, -1.2543367e-01,
        -1.9737554e-01, -2.2722632e-02, -6.9240725e-01,  7.2652334e-01],
       [ 4.3692350e-01, -1.9706452e+00, -1.6642106e-01,  5.4862052e-02,
        -8.3791959e-01, -1.3239880e-01, -9.9567705e-01,  9.4124246e-01],
       [-8.7577540e-01,  1.8741661e+00, -9.4874996e-01, -9.6571848e-02,
        

In [12]:
%%time
# 训练集和测试集都分20
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // 10
test_steps_per_shard = 5170 // batch_size // 10

output_dir = "generate_tfrecords"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard, None)
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, 10, valid_steps_per_shard, None)
test_tfrecord_fielnames = csv_dataset_to_tfrecords(
    test_basename, test_set, 10, test_steps_per_shard, None)
#执行会发现目录下总计生成了60个文件,这里文件数目改为一致，为了对比时间

CPU times: user 35 s, sys: 9.33 s, total: 44.3 s
Wall time: 34 s


In [13]:
!ls -l generate_tfrecords

总用量 1960
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00000-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00001-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00002-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00003-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00004-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00005-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00006-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00007-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00008-of-00010
-rw-rw-r-- 1 luke luke 47616 May  3 11:29 test_00009-of-00010
-rw-rw-r-- 1 luke luke 53568 May  3 11:29 train_00000-of-00020
-rw-rw-r-- 1 luke luke 53568 May  3 11:29 train_00001-of-00020
-rw-rw-r-- 1 luke luke 53568 May  3 11:29 train_00002-of-00020
-rw-rw-r-- 1 luke luke 53568 May  3 11:29 train_00003-of-00020
-rw-rw-r-- 1 luke luke 53568 May  3 11:29 train_00004-of-00020
-rw-rw-r-- 1 luke luke 53568 May  3 11:29 train_00005-of

In [33]:
#生成一下压缩的
# n_shards = 20
# train_steps_per_shard = 11610 // batch_size // n_shards
# valid_steps_per_shard = 3880 // batch_size // n_shards
# test_steps_per_shard = 5170 // batch_size // n_shards

# output_dir = "generate_tfrecords_zip"
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)

# train_basename = os.path.join(output_dir, "train")
# valid_basename = os.path.join(output_dir, "valid")
# test_basename = os.path.join(output_dir, "test")
# #只需修改参数的类型即可
# train_tfrecord_filenames = csv_dataset_to_tfrecords(
#     train_basename, train_set, n_shards, train_steps_per_shard,
#     compression_type = "GZIP")
# valid_tfrecord_filenames = csv_dataset_to_tfrecords(
#     valid_basename, valid_set, n_shards, valid_steps_per_shard,
#     compression_type = "GZIP")
# test_tfrecord_fielnames = csv_dataset_to_tfrecords(
#     test_basename, test_set, n_shards, test_steps_per_shard,
#     compression_type = "GZIP")

In [34]:
!ls -l generate_tfrecords_zip

总用量 860
-rw-rw-r-- 1 luke luke 10171 May  7 11:16 test_00000-of-00020
-rw-rw-r-- 1 luke luke 10230 May  7 11:16 test_00001-of-00020
-rw-rw-r-- 1 luke luke 10204 May  7 11:16 test_00002-of-00020
-rw-rw-r-- 1 luke luke 10213 May  7 11:16 test_00003-of-00020
-rw-rw-r-- 1 luke luke 10229 May  7 11:16 test_00004-of-00020
-rw-rw-r-- 1 luke luke 10200 May  7 11:16 test_00005-of-00020
-rw-rw-r-- 1 luke luke 10199 May  7 11:16 test_00006-of-00020
-rw-rw-r-- 1 luke luke 10215 May  7 11:16 test_00007-of-00020
-rw-rw-r-- 1 luke luke 10179 May  7 11:16 test_00008-of-00020
-rw-rw-r-- 1 luke luke 10149 May  7 11:16 test_00009-of-00020
-rw-rw-r-- 1 luke luke 10141 May  7 11:16 test_00010-of-00020
-rw-rw-r-- 1 luke luke 10221 May  7 11:16 test_00011-of-00020
-rw-rw-r-- 1 luke luke 10209 May  7 11:16 test_00012-of-00020
-rw-rw-r-- 1 luke luke 10214 May  7 11:16 test_00013-of-00020
-rw-rw-r-- 1 luke luke 10212 May  7 11:16 test_00014-of-00020
-rw-rw-r-- 1 luke luke 10209 May  7 11:16 test_00015-of-00020


In [14]:
#打印一下文件名
pprint.pprint(train_tfrecord_filenames)
pprint.pprint(valid_tfrecord_filenames)
pprint.pprint(test_tfrecord_fielnames)

['generate_tfrecords/train_00000-of-00020',
 'generate_tfrecords/train_00001-of-00020',
 'generate_tfrecords/train_00002-of-00020',
 'generate_tfrecords/train_00003-of-00020',
 'generate_tfrecords/train_00004-of-00020',
 'generate_tfrecords/train_00005-of-00020',
 'generate_tfrecords/train_00006-of-00020',
 'generate_tfrecords/train_00007-of-00020',
 'generate_tfrecords/train_00008-of-00020',
 'generate_tfrecords/train_00009-of-00020',
 'generate_tfrecords/train_00010-of-00020',
 'generate_tfrecords/train_00011-of-00020',
 'generate_tfrecords/train_00012-of-00020',
 'generate_tfrecords/train_00013-of-00020',
 'generate_tfrecords/train_00014-of-00020',
 'generate_tfrecords/train_00015-of-00020',
 'generate_tfrecords/train_00016-of-00020',
 'generate_tfrecords/train_00017-of-00020',
 'generate_tfrecords/train_00018-of-00020',
 'generate_tfrecords/train_00019-of-00020']
['generate_tfrecords/valid_00000-of-00010',
 'generate_tfrecords/valid_00001-of-00010',
 'generate_tfrecords/valid_00002

In [15]:
%%time
#把数据读取出来
expected_features = {
    "input_features": tf.io.FixedLenFeature([8], dtype=tf.float32),
    "label": tf.io.FixedLenFeature([1], dtype=tf.float32)
}

def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example,
                                         expected_features)
    return example["input_features"], example["label"]

def tfrecords_reader_dataset(filenames, n_readers=5,
                             batch_size=32, n_parse_threads=5,
                             shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
#         lambda filename: tf.data.TFRecordDataset(
#             filename, compression_type = "GZIP"),
          lambda filename: tf.data.TFRecordDataset(
            filename),
          cycle_length = n_readers
    )
    #洗牌，就是给数据打乱
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_example,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)  #原来写进去是一条一条的sample，要分配
    return dataset

#测试一下，tfrecords_reader_dataset是否可以正常运行
# tfrecords_train = tfrecords_reader_dataset(train_tfrecord_filenames,
#                                            batch_size = 3)
# for x_batch, y_batch in tfrecords_train.take(10):
#     print(x_batch)
#     print(y_batch)

CPU times: user 36 µs, sys: 0 ns, total: 36 µs
Wall time: 41.7 µs


In [16]:
%%time
#得到dataset,dataset是tensor，可以直接拿tensor训练

batch_size = 32
tfrecords_train_set = tfrecords_reader_dataset(
    train_tfrecord_filenames, batch_size = batch_size)
tfrecords_valid_set = tfrecords_reader_dataset(
    valid_tfrecord_filenames, batch_size = batch_size)
tfrecords_test_set = tfrecords_reader_dataset(
    test_tfrecord_fielnames, batch_size = batch_size)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function tfrecords_reader_dataset.<locals>.<lambda> at 0x7f2844749620>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <function tfrecords_reader_dataset.<locals>.<lambda> at 0x7f2844749620>. Note that functions defi

In [17]:
type(tfrecords_train_set)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [18]:
for i in tfrecords_train_set.take(1):
    print(i)

(<tf.Tensor: shape=(32, 8), dtype=float32, numpy=
array([[ 4.85305160e-01, -8.49241912e-01, -6.53012618e-02,
        -2.33796556e-02,  1.49743509e+00, -7.79065788e-02,
        -9.02363241e-01,  7.81451464e-01],
       [ 1.11519516e-01,  4.32361901e-01, -5.00933290e-01,
        -3.21237385e-01, -1.15819156e+00,  2.72315852e-02,
         7.86616385e-01, -1.16100752e+00],
       [-6.97008371e-01, -6.89041436e-01,  3.55311275e-01,
         3.80839527e-01, -7.09084928e-01, -6.55453503e-02,
         1.58911502e+00, -1.64038050e+00],
       [ 4.95588928e-01, -7.69141674e-01, -1.79629475e-01,
        -1.61498353e-01, -4.26011652e-01, -3.83407474e-02,
         9.59246933e-01, -1.42566133e+00],
       [-3.21411103e-01,  8.32863092e-01,  1.33539941e-02,
        -2.07708240e-01, -3.38912189e-01,  3.30382772e-02,
        -8.23046565e-01,  6.56614780e-01],
       [-1.11997497e+00, -1.32984328e+00,  1.41900450e-01,
         4.65813696e-01, -1.03017777e-01, -1.07441843e-01,
        -7.95052409e-01,  1

In [19]:
#开始训练
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=[8]),
    keras.layers.Dense(1),
])
model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

history = model.fit(tfrecords_train_set,
                    validation_data = tfrecords_valid_set,
                    steps_per_epoch = 11160 // batch_size,
                    validation_steps = 3870 // batch_size,
                    epochs = 100,
                    callbacks = callbacks)

Epoch 1/100
348/348 [==============================] - 1s 2ms/step - loss: 0.8290 - val_loss: 1.4444
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 2.2042 - val_loss: 0.5141
Epoch 3/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4487 - val_loss: 0.4691
Epoch 4/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4077 - val_loss: 0.4416
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3934 - val_loss: 0.4212
Epoch 6/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3769 - val_loss: 0.4152
Epoch 7/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3744 - val_loss: 0.4132
Epoch 8/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3556 - val_loss: 0.3951
Epoch 9/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3580 - val_loss: 0.4157
Epoch 10/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3458 - val_lo

In [20]:
model.evaluate(tfrecords_test_set, steps = 5160 // batch_size)

161/161 [==============================] - 0s 2ms/step - loss: 0.3376


0.33755674958229065